In [2]:
api_key='RGAPI-473c5960-3ed2-4d30-822d-3c0861908f09'
import requests

In [3]:
import pandas as pd

root= 'https://euw1.api.riotgames.com/'
challenger_root = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
grandmaster_root = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
master_root = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

challenger_response = requests.get(root+challenger_root+'?api_key='+api_key)
grandmaster_response = requests.get(root+grandmaster_root+'?api_key='+api_key)
master_response = requests.get(root+master_root+'?api_key='+api_key)

challenger_df = pd.DataFrame(challenger_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
grandmaster_df=pd.DataFrame(grandmaster_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
master_df=pd.DataFrame(master_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)


#concatenating the dataframes
high_rank_ladder=pd.concat([challenger_df, grandmaster_df, master_df])[0:100].reset_index(drop=True)
high_rank_ladder=high_rank_ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
high_rank_ladder['rank']+=1

high_rank_ladder

,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,5NzokD2VZC4DB3qBJdl6LOO3PiRQcORDRHZRLgSKdBtt0YA,1580,228,141,True,False,False,False
1,2,KFCNaCBCxDEfbGcW3ltdr_of4XStG8Q1CLDY6OOFTcm1Kz0,1539,261,173,True,False,False,True
2,3,JGhpg_EeM7jdF-U90nyputqFS-hC3ftrDN---vtbuvo7uDzA,1491,239,162,True,False,False,False
3,4,U4_ndlbviSSK2g20hBy5NNOG1awHFhM2iV_u4NlV73ayAd...,1454,451,358,True,False,False,False
4,5,9lgPXUXCgFimXrAUcRSIajMSS5yweHgAliPr-_H_CumwWGY,1412,155,74,True,False,False,False
...,...,...,...,...,...,...,...,...,...
95,96,kmfgO3lne2VDx6qTJLrnJDM-MdVRq5-VtXyrrJ8CC9agnnk,1008,310,252,False,False,False,True
96,97,e-iWepKYb0A9OrA6ruwpEVRruVpkdg40hAYhAM231jVuZR0F,1008,211,154,False,False,False,True
97,98,pfge3KjI03Y23qgBjjlts-2KS6h46oMUXipSMrzMlnasgtk,1008,306,250,False,False,False,True
98,99,3LQpZ9VEpbwq69aT52H-9Gy9TGq_k_Y-7Ar-Rcv8EnS7T0cu,1008,332,270,True,False,False,False


In [4]:
import time
def get_puuid_from_summoner_id(summonerId=None):
    link=f'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}'

    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        puuid=response.json()['puuid']
        return puuid

In [4]:
puuid_list = []

for summoner in high_rank_ladder['summonerId']:
    puuids = get_puuid_from_summoner_id(summonerId=summoner)
    puuid_list.append(puuids)

rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded


In [7]:
puuid_list[0:5]

['buvMukA-JVre-d3tH8WGdH0Fewk5VwFKmuVog33gO7PDRcf_-BV4Iy6jCV0DsaN_HfPU1Or2suJVSA',
 'rH_mM7PpbVDWYWZeAY9_wh_CKd4_lTUbS8dMpMRxfOvZDNp_wQVAG_KGOORmnlDhViUhIyNIuES0hQ',
 'aZFgbLH6q7UTdHCTLmcBIvarGyRb38eEzW0b_jwnmFx90Ur988DDZvv9kSyoqOedO_rIM5b6Y2VEgg',
 'A4-eBTYpPDIpHuBu2ChYaB7Jq2Ja5YXAW5LBwBjVjljKnmv3kS98yPQXotPL6tURenUmBhUJplH4mA',
 'GFaoHL-BXl_WQ7ni6OoBY4PI8vIaPFsLUo5EH0WN5QNX-j63OFSF79byTEhwHsxqsl31Ifpt7AcqwQ']

In [5]:
def get_match_id_from_puuid(puuid=None):
    """
    Function to retrieve the most recent 20 match id's. Takes a player's puuid as argument and outputs the match id's of their most recent 20 ranked matches.
    Queue and type must remain as 420 and ranked respectively as this will only get the ranked matches which is what we want.
    """
    link=f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&type=ranked&start=0&count=15&api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        match_id_list=response.json()
        break
    return match_id_list


In [7]:
all_match_id_list=[]

for player_puuid in puuid_list:
    match_ids=get_match_id_from_puuid(puuid=player_puuid)
    all_match_id_list.extend(match_ids)

rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded


In [10]:
all_match_id_list[0:5]

['EUW1_6928661440',
 'EUW1_6928550011',
 'EUW1_6928471250',
 'EUW1_6927403998',
 'EUW1_6926350115']

In [8]:
len(all_match_id_list)

15000

In [13]:
from collections import Counter

counter = Counter(all_match_id_list)
duplicates = [item for item, count in counter.items() if count > 1]
if duplicates:
    print("Duplicates found:", duplicates)
    print(len(duplicates))



Duplicates found: ['EUW1_6928661440', 'EUW1_6928550011', 'EUW1_6928471250', 'EUW1_6927403998', 'EUW1_6926350115', 'EUW1_6925234085', 'EUW1_6923994009', 'EUW1_6922507359', 'EUW1_6921497820', 'EUW1_6920157556', 'EUW1_6915449614', 'EUW1_6914203442', 'EUW1_6929836644', 'EUW1_6929780661', 'EUW1_6929720013', 'EUW1_6929657168', 'EUW1_6929605118', 'EUW1_6929586231', 'EUW1_6929264093', 'EUW1_6928512060', 'EUW1_6927617479', 'EUW1_6927582701', 'EUW1_6927521342', 'EUW1_6927472237', 'EUW1_6927416111', 'EUW1_6927256828', 'EUW1_6929512909', 'EUW1_6929486741', 'EUW1_6928779209', 'EUW1_6928291774', 'EUW1_6923501139', 'EUW1_6923418997', 'EUW1_6921387114', 'EUW1_6915740520', 'EUW1_6915618334', 'EUW1_6915457903', 'EUW1_6915394465', 'EUW1_6914335508', 'EUW1_6914167298', 'EUW1_6914130160', 'EUW1_6929648620', 'EUW1_6929604501', 'EUW1_6926231476', 'EUW1_6924112572', 'EUW1_6924050763', 'EUW1_6923820363', 'EUW1_6923685124', 'EUW1_6920058129', 'EUW1_6919935575', 'EUW1_6919839293', 'EUW1_6917422399', 'EUW1_691463

# Removing duplicates from match lists (they are the same matches)

In [14]:
from collections import Counter

# Count the occurrences of each match ID
match_id_counts = Counter(all_match_id_list)

# Find the duplicate match IDs
duplicate_match_ids = [match_id for match_id, count in match_id_counts.items() if count > 1]

# Remove duplicate match IDs from the list
unique_match_id_list = list(set(all_match_id_list))

print(f"Original number of match IDs: {len(all_match_id_list)}")
print(f"Number of duplicate match IDs: {len(duplicate_match_ids)}")
print(f"Number of unique match IDs after removal: {len(unique_match_id_list)}")


Original number of match IDs: 15000
Number of duplicate match IDs: 3218
Number of unique match IDs after removal: 6927


In [13]:
unique_match_id_list[0:5]

['EUW1_6926030077',
 'EUW1_6925891806',
 'EUW1_6912141732',
 'EUW1_6905212859',
 'EUW1_6922441165']

# Retrieivng entire match JSON given a match ID

In [6]:
def get_match_info_from_id(matchId=None):
    link= f'https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        elif response.status_code==200:
            break
        elif response.status_code==404:
            return 0
        else:
            print('Response is bad: '+str(response.status_code))
    match_data=response.json()
    return match_data

# Retrieving specific match information that I am interested in

In [7]:
import pandas as pd

def get_specific_match_info(data):
    info = data['info']
    metadata = data['metadata']
    participants = metadata['participants']  # List of player puuids
    teams = info['teams']
    
    #specific stats that applies to every player in the match
    game_duration = info['gameDuration']
    game_ended_in_surrender = info['participants'][0]['gameEndedInSurrender']
    game_ended_in_early_surrender=info['participants'][0]['gameEndedInEarlySurrender']
    blue_side_won = info['participants'][0]['win']
    

    #objectives will be considered separately because its in teamsDTO and not participantDTO

    #blue side objectives
    blue_side_tower = teams[0]['objectives']['tower']['kills']
    blue_side_dragon = teams[0]['objectives']['dragon']['kills']
    blue_side_baron = teams[0]['objectives']['baron']['kills']
    blue_side_riftHerald = teams[0]['objectives']['riftHerald']['kills']
    blue_side_horde = teams[0]['objectives']['horde']['kills']
    blue_side_inhibitor = teams[0]['objectives']['inhibitor']['kills']

    #red side objectives
    red_side_tower = teams[1]['objectives']['tower']['kills']
    red_side_dragon = teams[1]['objectives']['dragon']['kills']
    red_side_baron =teams[1]['objectives']['baron']['kills']
    red_side_riftHerald =teams[1]['objectives']['riftHerald']['kills']
    red_side_horde = teams[1]['objectives']['horde']['kills']
    red_side_inhibitor = teams[1]['objectives']['inhibitor']['kills']


    #build dictionaries for both blue adn red sides stats
    blue_side_stats = {
        'minions_killed': 0,
        'kills': 0,
        'deaths': 0,
        'assists': 0,
        'gold_earned': 0,
        'gold_spent': 0,
        'wards_killed':0,
        'wards_placed':0,
        'vision_score':0
    }

    red_side_stats = {
        'minions_killed': 0,
        'kills': 0,
        'deaths': 0,
        'assists': 0,
        'gold_earned': 0,
        'gold_spent': 0,
        'wards_killed':0,
        'wards_placed':0,
        'vision_score':0
    }
    
    
    #iterate through each players data. if the index of their participant id/puuid is less than 5 then they are on the blue team, otherwise they are on red team
    #side_stats is therefore assigned to each side depending on whether the participant/player id (i) is less than or greater than 5.
    for i, player in enumerate(info['participants']):
        if i < 5:
            side_stats=blue_side_stats
        else:
            side_stats=red_side_stats

        neautral_minions_killed = player['neutralMinionsKilled']
        minions_killed = player['totalMinionsKilled']
        kills = player['kills']
        assists = player['assists']
        deaths = player['deaths']
        gold_earned = player['goldEarned']
        gold_spent = player['goldSpent']
        wards_killed = player['wardsKilled']
        wards_placed=player['wardsPlaced']
        vision_score=player['visionScore']

        #add up the stats for both blue and red sides
        side_stats['minions_killed'] += neautral_minions_killed+minions_killed
        side_stats['assists'] += assists
        side_stats['deaths'] += deaths
        side_stats['kills'] += kills
        side_stats['gold_earned']+=gold_earned
        side_stats['gold_spent']+=gold_spent
        side_stats['wards_killed']+=wards_killed
        side_stats['wards_placed']+=wards_placed
        side_stats['vision_score']+=vision_score

    #create a dataframe with blue and red side stats
    df = pd.DataFrame({
        #game info
        'game_duration':[game_duration],
        'game_ended_in_surrender': [game_ended_in_surrender],
        'game_ended_in_early_surrender': [game_ended_in_early_surrender],
        'blue_side_won': [blue_side_won],

        #blue side
        'total_blue_side_minions_killed': [blue_side_stats['minions_killed']],
        'total_blue_side_kills': [blue_side_stats['kills']],
        'total_blue_side_deaths': [blue_side_stats['deaths']],
        'total_blue_side_assists': [blue_side_stats['assists']],
        'total_blue_side_gold_earned':[blue_side_stats['gold_earned']],
        'total_blue_side_gold_spent':[blue_side_stats['gold_spent']],
        'total_blue_side_wards_killed':[blue_side_stats['wards_killed']],
        'total_blue_side_wards_placed':[blue_side_stats['wards_placed']],
        'total_blue_side_vision_score':[blue_side_stats['vision_score']],
        'total_towers_destroyed_by_blue': [blue_side_tower],
        'total_inhibitors_destroyed_by_blue': [blue_side_inhibitor],
        'total_blue_side_dragons_killed': [blue_side_dragon],
        'total_blue_side_horde': [blue_side_horde],
        'total_blue_side_baron':[blue_side_baron],
        'total_blue_side_riftHerald':[blue_side_riftHerald],

        #red side
        'total_red_side_minions_killed': [red_side_stats['minions_killed']],
        'total_red_side_kills': [red_side_stats['kills']],
        'total_red_side_deaths': [red_side_stats['deaths']],
        'total_red_side_assists': [red_side_stats['assists']],
        'total_red_side_gold_earned':[red_side_stats['gold_earned']],
        'total_red_side_gold_spent':[red_side_stats['gold_spent']],
        'total_red_side_ward_takedowns':[red_side_stats['wards_killed']],
        'total_red_side_wards_placed':[red_side_stats['wards_placed']],
        'total_red_side_vision_score':[red_side_stats['vision_score']],
        'total_towers_destroyed_by_red': [red_side_tower],
        'total_inhibitors_destroyed_by_red': [red_side_inhibitor],
        'total_red_side_dragons_killed': [red_side_dragon],
        'total_red_side_horde': [red_side_horde],
        'total_red_side_baron':[red_side_baron],
        'total_red_side_riftHerald':[red_side_riftHerald]

    })

    return df



In [8]:
get_match_info_from_id(matchId='EUW1_6922507359')

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_6922507359',
  'participants': ['zuAncw-Y2V8kOFq2ooZOFPx_BuDOyLD87p269DcyXrf44FoVfvbDDNHhhTKoP5d8ar8pMw5nzyPngg',
   'V6fMOp5_6uJzRr9zzV_OcjcolGb1u6AWMihiHauRCAKzLrCt0ysQI_9gC4osyAJ8s59JACNW_6Lf1g',
   'tFHLuq5c7F5IcfA4tKABSX5af8HLgRLEjEzyGCRLzaENRAIrmwRjfHceDOCrqLDVyYt2XwykRx8Vig',
   'PLvk6LWDB_4bdKxulfvyk5GFTNaMJIthGs-yPpW7qIqlBUF52B1mjkTuW5ngk6PFSiJ8LD7Mc-aGbQ',
   'qrdGXPjGFtt9EH-EkyfCCKE-3T-eDKBeKT26KnQPIA5QSn5f1UM6fDK3kMRpj8vDML5K3FJsaRQ_cA',
   'sOCKEt2sR01YP9W28bXwTPn7gqOnZs3_q9ZZiRBrFzqggaFnxy4cv5A9xdTULMmYOGxtCNtjdft3dw',
   'buvMukA-JVre-d3tH8WGdH0Fewk5VwFKmuVog33gO7PDRcf_-BV4Iy6jCV0DsaN_HfPU1Or2suJVSA',
   'g0ZhqegHwAgkEkbP6nl2ctb0cDX344SKDFtFXQmB1HoOgGgLHZtZ3F3Qx9W6Cwq3pdMmcxTynQEUug',
   'zRqBRkxGsLavYAtZTu5u7vnb3lYfzD0YrpDUWbEHO0rd7evpNAek4109MIbNNZOMP2ARfAFEtvSmWA',
   'q6etrDTBhLZ_keTEP9IxenZzlQMsLYneHQa6qYoitTgrkOKE7JMVra9sKMaXMGOr22waH1kLXA9MOA']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 171

In [17]:
match_ids = unique_match_id_list

df=pd.DataFrame()

for match_id in match_ids:
    game = get_match_info_from_id(matchId=match_id)
    matchdf=get_specific_match_info(game)

    df= pd.concat([df,matchdf])

rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded


In [19]:
df.to_csv('final_high_rank_data.csv')